In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class TextModel(nn.Module):
    def __init__(self, text_input_dim, hidden_dim):
        super(TextModel, self).__init__()
        self.fc = nn.Linear(text_input_dim, hidden_dim)
    
    def forward(self, x):
        out = self.fc(x)
        return out

class ImageModel(nn.Module):
    def __init__(self, image_input_dim, hidden_dim):
        super(ImageModel, self).__init__()
        self.fc = nn.Linear(image_input_dim, hidden_dim)
    
    def forward(self, x):
        out = self.fc(x)
        return out

class FusionModel(nn.Module):
    def __init__(self, hidden_dim, num_classes):
        super(FusionModel, self).__init__()
        self.fc = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, text_features, image_features):
        fused_features = torch.cat((text_features, image_features), dim=1)
        out = self.fc(fused_features)
        return out

# 定义模型参数
text_input_dim = 300  # 假设文本特征维度为300
image_input_dim = 512  # 假设图像特征维度为512
hidden_dim = 256  # 隐层维度
num_classes = 3  # 类别数量：positive, neutral, negative

text_model = TextModel(text_input_dim, hidden_dim)
image_model = ImageModel(image_input_dim, hidden_dim)
fusion_model = FusionModel(hidden_dim*2, num_classes)


In [2]:

import torch.nn as nn
import torch.optim as optim
def get_text_features(guid):
    # 假设您使用的是预训练的词向量模型（如GloVe）来表示文本
    # 这里仅作示例，实际上需要根据您的具体情况进行修改
    text_file_path = "data/{}.txt".format(guid)
    with open(text_file_path, "r", encoding="latin-1") as file:
        text_data = file.read()
    
    # 在这里进行文本特征提取的相关操作，例如文本预处理、词嵌入等
    # 这里使用随机向量作为示例
    random_text_features = torch.randn(1, 300)  # 假设文本特征维度为300
    
    return random_text_features

def get_image_features(guid):
    # 假设使用的是预训练的卷积神经网络（如ResNet）来提取图像特征
    image_file_path = "data/{}.jpg".format(guid)
    
    # 在这里进行图像特征提取的相关操作，例如图像预处理、CNN网络等
    # 这里使用随机向量
    random_image_features = torch.randn(1, 512)  # 假设图像特征维度为512
    
    return random_image_features


# 定义训练数据和标签
# train_data = [("4597", "negative"), ("2020", "positive"), ("4784", "neutral")]  # 根据实际数据填充

file_train = "train.txt"

# 读取文件内容
with open(file_train, "r") as file:
    lines = file.readlines()[1:]
    
# 构建训练数据列表
train_data = []
for line in lines:
    # 解析每一行数据
    guid, tag = line.strip().split(",")
    train_data.append((guid, tag))


# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(text_model.parameters()) + list(image_model.parameters()) + list(fusion_model.parameters()), lr=0.001)

correct_predictions = 0
total_predictions = 0

# 训练模型
for epoch in range(20):  # 假设训练轮数为10
    running_loss = 0.0
    for data in train_data:
        guid, tag = data
        
        # 假设文本特征提取函数为get_text_features，图像特征提取函数为get_image_features
        text_features = get_text_features(guid)  
        image_features = get_image_features(guid)
        
        # 将标签转换为对应的数字编码
        if tag == "positive":
            label = 0
        elif tag == "neutral":
            label = 1
        else:
            label = 2
        
        # 清零梯度、前向传播、计算损失、反向传播、更新参数
        optimizer.zero_grad()
        text_output = text_model(text_features)
        image_output = image_model(image_features)
        fusion_output = fusion_model(text_output, image_output)
        loss = criterion(fusion_output, torch.tensor([label]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted_label = torch.max(fusion_output, dim=1)
        correct_predictions += (predicted_label == label).sum().item()
        total_predictions += 1
    
    accuracy = correct_predictions / total_predictions
    
    # 打印每个epoch的损失
    print("Epoch {}, Loss: {}, Accuracy: {}".format(epoch+1, running_loss/len(train_data), accuracy))


Epoch 1, Loss: 1.1431731250626036, Accuracy: 0.473
Epoch 2, Loss: 1.192255952037056, Accuracy: 0.482625
Epoch 3, Loss: 1.2430010397074511, Accuracy: 0.4820833333333333
Epoch 4, Loss: 1.2922216650745249, Accuracy: 0.4831875
Epoch 5, Loss: 1.3327730553822474, Accuracy: 0.4846
Epoch 6, Loss: 1.3764226371102268, Accuracy: 0.483625
Epoch 7, Loss: 1.390814856598401, Accuracy: 0.48342857142857143
Epoch 8, Loss: 1.4477200187352572, Accuracy: 0.48221875
Epoch 9, Loss: 1.452776273423471, Accuracy: 0.4821944444444444
Epoch 10, Loss: 1.4726631617526436, Accuracy: 0.48125
Epoch 11, Loss: 1.4866633351401743, Accuracy: 0.48163636363636364
Epoch 12, Loss: 1.5299565114100477, Accuracy: 0.4805625
Epoch 13, Loss: 1.5383177410118807, Accuracy: 0.48007692307692307
Epoch 14, Loss: 1.5498667892888889, Accuracy: 0.4803928571428571
Epoch 15, Loss: 1.564124803507959, Accuracy: 0.47978333333333334
Epoch 16, Loss: 1.612954347028608, Accuracy: 0.47875
Epoch 17, Loss: 1.5881462660410859, Accuracy: 0.478205882352941

In [3]:
# 定义测试数据
# test_data = [("8", "null"), ("259", "null"), ("3216", "null")]  # 数据填充
# 定义文件路径
file_path = "test_without_label.txt"

# 读取文件内容
with open(file_path, "r") as file:
    lines = file.readlines()[1:]
    
# 构建测试数据列表
test_data = []
for line in lines:
    # 解析每一行数据
    guid, tag = line.strip().split(",")
    test_data.append((guid, tag))
    

# 预测测试集情感标签
predictions = []
for data in test_data:
    guid, _ = data
    
    # 文本特征提取函数为get_text_features，图像特征提取函数为get_image_features
    text_features = get_text_features(guid)
    image_features = get_image_features(guid)
    
    text_output = text_model(text_features)
    image_output = image_model(image_features)
    fusion_output = fusion_model(text_output, image_output)
    
    _, predicted_label = torch.max(fusion_output, dim=1)
    
    # 将数字编码转换回标签
    if predicted_label == 0:
        predicted_tag = "positive"
    elif predicted_label == 1:
        predicted_tag = "neutral"
    else:
        predicted_tag = "negative"
    
    predictions.append((guid, predicted_tag))

# 将预测结果写入测试集文件
with open("test_with_label.txt", "w") as file:
    file.write("guid,tag\n")
    for prediction in predictions:
        guid, tag = prediction
        file.write("{},{}\n".format(guid, tag))